In [1]:
from sentinelsat import SentinelAPI
from shapely.geometry import Point, Polygon
import dask 
import matplotlib.pyplot as plt
import zipfile
import geopandas as gpd
import threading
import folium
import rasterio as rio
import os

import numpy as np
np.seterr(divide='ignore', invalid='ignore')

from src.dumper import Santinel2Dumper
from src.manager import Santinel2Manager
from src.misc import MapManager


REDOWNLOAD = False

DOWNLOAD_PATH="./satelite_datasets"
RESULTS_PATH = "./satelite_results"



## Set and view the chose target data

In [2]:
# create target poligon
# coordonates of forest near Medias,Sibiu
TARGET_SHAPE = [
    (24.197322, 46.202205), 
    (24.220021, 46.194055), 
    (24.261777, 46.179938),  
    (24.257313, 46.147835),
    (24.209952, 46.154057),
    (24.211586, 46.170317),
    (24.190559, 46.179364)
]


# display map
MapManager.get_map_with_target(TARGET_SHAPE)


## See what data is available 

In [3]:
# get query results 
dumper = Santinel2Dumper()

products_gdf = dumper.get_products_from_query(
    Polygon(TARGET_SHAPE),
    date_range=("20190930", "20201030")
)

products_gdf_sorted = products_gdf.sort_values(['beginposition'], ascending=[True])
products_gdf_sorted.columns

2021-02-24 23:21:23:    INFO: [sentinelsat.SentinelAPI]: Found 25 products


Index(['title', 'link', 'link_alternative', 'link_icon', 'summary', 'ondemand',
       'beginposition', 'endposition', 'ingestiondate', 'orbitnumber',
       'relativeorbitnumber', 'vegetationpercentage', 'notvegetatedpercentage',
       'waterpercentage', 'unclassifiedpercentage',
       'mediumprobacloudspercentage', 'highprobacloudspercentage',
       'snowicepercentage', 'cloudcoverpercentage', 'level1cpdiidentifier',
       'format', 'processingbaseline', 'platformname', 'filename',
       'instrumentname', 'instrumentshortname', 'size', 's2datatakeid',
       'producttype', 'platformidentifier', 'orbitdirection',
       'platformserialidentifier', 'processinglevel', 'identifier', 'uuid',
       'geometry'],
      dtype='object')

## Targets

In [4]:
# look for a close vegetationpercentage
targets = products_gdf_sorted.iloc[[0, -1]]
targets[["title","beginposition","summary","vegetationpercentage"]]

,title,beginposition,summary,vegetationpercentage
92f702c0-bcbf-41fc-ad92-6e7ee27e8c04,S2B_MSIL2A_20191001T092029_N0213_R093_T34TGS_2...,2019-10-01 09:20:29.024,"Date: 2019-10-01T09:20:29.024Z, Instrument: MS...",61.325228
acab8fba-5ee4-40d9-bf4d-529abee4f1bc,S2A_MSIL2A_20201020T092031_N0214_R093_T34TGS_2...,2020-10-20 09:20:31.025,"Date: 2020-10-20T09:20:31.025Z, Instrument: MS...",66.946751


## Download the query results

In [5]:
if REDOWNLOAD:
    dumper.download_satelite_data(targets.iloc[ 0].uuid,DOWNLOAD_PATH)
    dumper.download_satelite_data(targets.iloc[-1].uuid,DOWNLOAD_PATH)
else:
    print("Redownload is set to false")


Redownload is set to false


## Assamble data from old imagery

In [6]:
manager_old = Santinel2Manager(f"{DOWNLOAD_PATH}/{targets.iloc[0].title}.zip")

2021-02-24 23:21:23:    INFO: [Santinel2Manager]: Reading dataset from: ./satelite_datasets/S2B_MSIL2A_20191001T092029_N0213_R093_T34TGS_20191001T133427.zip


In [7]:
%%time
THRESHOLD = .7
_meta = manager_old.get_R10m_reader().meta
old_img_NDVI = manager_old.get_NDVI()
old_img_NDVI_veg = manager_old.get_NDVI_vegetation_only(THRESHOLD)

2021-02-24 23:21:23:    INFO: [Santinel2Manager]: Getting R10m for ./satelite_datasets/S2B_MSIL2A_20191001T092029_N0213_R093_T34TGS_20191001T133427.zip
2021-02-24 23:21:23:    INFO: [Santinel2Manager]: Assambling NDVI image!
2021-02-24 23:21:23:    INFO: [Santinel2Manager]: Getting R10m for ./satelite_datasets/S2B_MSIL2A_20191001T092029_N0213_R093_T34TGS_20191001T133427.zip
2021-02-24 23:21:33:    INFO: [Santinel2Manager]: Finnished assambling NDVI image!


TypeError: get_NDVI_vegetation_only() missing 1 required positional argument: 'threshold'

In [ ]:
%%time

old_manager_old.write_image(old_img_NDVI,_meta,f"{RESULTS_PATH}/{targets.iloc[0].title}/NDVI.tiff")
old_manager_old.write_image(old_img_NDVI_veg,_meta,f"{RESULTS_PATH}/{targets.iloc[0].title}/NDVI_veg.tiff")



In [ ]:
%%time

manager_old.crop_image(
    f"{RESULTS_PATH}/{targets.iloc[0].title}/NDVI.tiff",
    f"{RESULTS_PATH}/{targets.iloc[0].title}/cropped_NDVI.tif",
    Polygon(TARGET_SHAPE)
)
manager_old.crop_image(
    f"{RESULTS_PATH}/{targets.iloc[0].title}/NDVI_veg.tiff",
    f"{RESULTS_PATH}/{targets.iloc[0].title}/cropped_NDVI_veg.tif",
    Polygon(TARGET_SHAPE)
)

## Assamble data from new imagery

In [ ]:
manager_new = Santinel2Manager(f"{DOWNLOAD_PATH}/{targets.iloc[-1].title}.zip")

In [ ]:
%%time

_meta = manager_new.get_R10m_reader().meta
new_img_NDVI = manager_new.get_NDVI()
new_img_NDVI_veg = manager_new.get_NDVI_vegetation_only()

In [ ]:
%%time

manager_new.write_image(new_img_NDVI,_meta,f"{RESULTS_PATH}/{targets.iloc[-1].title}/NDVI.tiff")
manager_new.write_image(new_img_NDVI_veg,_meta,f"{RESULTS_PATH}/{targets.iloc[-1].title}/NDVI_veg.tiff")

In [ ]:
%%time

manager_new.crop_image(
    f"{RESULTS_PATH}/{targets.iloc[-1].title}/NDVI.tiff",
    f"{RESULTS_PATH}/{targets.iloc[-1].title}/cropped_NDVI.tif",
    Polygon(TARGET_SHAPE)
)
manager_new.crop_image(
    f"{RESULTS_PATH}/{targets.iloc[-1].title}/NDVI_veg.tiff",
    f"{RESULTS_PATH}/{targets.iloc[-1].title}/cropped_NDVI_veg.tif",
    Polygon(TARGET_SHAPE)
)